In [ ]:
# Example 22: live observation of single satellite 
# SPDX-FileCopyrightText: Copyright (C) 2023 Andreas Naber <annappo@web.de>
# SPDX-License-Identifier: GPL-3.0-only

%matplotlib widget

import asyncio
from rtlsdr import RtlSdr
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np
from scipy.fft import fft, ifft
import gpslib_tutorial as gpslib
import time
import datetime
import json
from collections import deque
import linecache
import sys

# -------- floating point types used in arrays ---------

MY_FLOAT = np.float32
MY_COMPLEX = np.complex64


# --------- global variables -------------------

SAMPLE_RATE = 2.048E6
SDR_FREQCORR = 0
SDR_CENTERFREQ = 1575.42e6
SDR_GAIN = 49.9

MEAS_RUNNING = True       
STATUS_MSG = ''           

SMP_TIME = 0              
    
N_CYC = 32   
CODE_SAMPLES = 2048 
NGPS = N_CYC*CODE_SAMPLES
CORR_AVG = 8             
DLF = 1024 // N_CYC      
MAXBUFSIZE = 8*NGPS      

SEC_TIME = np.linspace(1,NGPS,NGPS,endpoint=True)/SAMPLE_RATE
FRAME_LIST = []     
COPH_LIST = []      

MIN_FREQ = -5000.0
MAX_FREQ = +5000.0
STEP_FREQ = 200            
START_FREQ = MIN_FREQ
CORR_MIN = 8     
    
SAVE_RESULTS = False
SAVE_DATE = '150624'

SAT_NO = 3
IT_SWEEP = 40                  # depends on computer speed
MEAS_TIME = 600                # time in second

# ------- Exception handling & Output -------------------

EXC = ''

def printException():
    global EXC
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    EXC = ('EXCEPTION IN ({}, LINE {} "{}"): {}'.\
           format(filename, lineno, line.strip(), exc_obj))        
       
        
# -------- keyboard events ---------------

def onKeypress(event):
    global STATUS_MSG
    global MEAS_RUNNING
    sys.stdout.flush()
    if event.key in ['q','Q']:
        MEAS_RUNNING = False
        STATUS_MSG = 'q pressed'

# ------- data buffer -----------------

MAXBUFSIZE = 8       
BUFFER = deque([],maxlen=MAXBUFSIZE)
NBUF = 0     
BUFSKIP = 0

def pushToBuffer(data):
    global BUFFER
    global NBUF
    global BUFSKIP

    if NBUF >= MAXBUFSIZE:
        BUFFER.clear()
        NBUF = 0
        BUFSKIP += MAXBUFSIZE
        
    BUFFER.append(data)
    NBUF += 1
    
    
def pullFromBuffer(n):
    global BUFFER
    global NBUF
    global BUFSKIP

    try:
        data = BUFFER.popleft()
        NBUF -= 1
        skip = BUFSKIP
        BUFSKIP = 0
    except IndexError:
        data = []
        skip = 0
    
    return data,n,skip


# -------- JSON Encoder for numpy ----------------------

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)
        
    
# ----------- Save Positions, Frames and CodePhases --------------------------------           
            
def saveResults(saveDate,satNo):                    
    try:
        with open(f'CP_{saveDate}_{satNo}.json','w') as file:
            json.dump(COPH_LIST,file,indent=2,cls=MyEncoder)
        with open(f'SF_{saveDate}_{satNo}.json','w') as file:
            json.dump(FRAME_LIST,file,indent=2,cls=MyEncoder)
                        
    except:
        printException() 
        
# ------------- plot text -----------

TOW_NO = -1
WEEK_NO = 0

def plotText(frameData):
    global TOW_NO
    global WEEK_NO
    if TOW_NO < 0:
        TOW_NO = frameData[-1]['tow']
    if frameData[-1]['ID'] == 1 and WEEK_NO == 0:
        WEEK_NO = frameData[-1]['weekNum']
        gpsTStr = gpslib.gpsTimeStr(TOW_NO,WEEK_NO)
        startL.set_text('START: %s' % (gpsTStr))
    for i,fd in enumerate(frameData[-MAXL:]):
        gpsT = gpslib.gpsTime(fd['tow'],0)
        dataL[i][keys[0]].set_text(gpsT.strftime('%H:%M:%S')) 
        dataL[i][keys[1]].set_text('%d' %(fd['tow']))
        dataL[i][keys[2]].set_text('%d' % (fd['ID']))
        dataL[i][keys[3]].set_text('%1.6f s' % (fd['ST']/SAMPLE_RATE))

# ------------- plot data -----------

def plotData(y1,y2):
    line1.set_ydata(y1.real)
    line2.set_ydata(y2)
    plt.draw() 
    
# ------- process data -------------------


async def processData():   
    global MEAS_RUNNING
    global SMP_TIME
    global FRAME_LIST
    global COPH_LIST
    
    gpsPlot = np.zeros(DLF*N_CYC,dtype=MY_COMPLEX)   
    bitPlot = np.zeros(DLF*N_CYC,dtype=MY_FLOAT)
    sweepAllFreq = True
    
    satStream = gpslib.SatStream(SAT_NO,START_FREQ,itSweep=IT_SWEEP)
    satStream.CALC_PLOT = True       # default is False 
    startSweep = True
    
    noStream = 0
    try:
        while MEAS_RUNNING:
            if NBUF >= 1:
                data,n,skip = pullFromBuffer(NGPS)
                SMP_TIME += NGPS + skip                   
                if sweepAllFreq:
                    sweepAllFreq,_,_,_ = satStream.process(data,SMP_TIME,
                                                           sweep=startSweep)
                    startSweep = False
                else:    
                    sweepAllFreq,frameData,codePhase,cpQ \
                        = satStream.process(data,SMP_TIME)
                    gpsData = satStream.GPSDATA
                    bitData = satStream.BITDATA
                    
                    if len(frameData)>0 and 'tow' in frameData[0]:
                        FRAME_LIST += frameData
                    gpsPlot = np.concatenate((gpsPlot[N_CYC:DLF*N_CYC],
                                              gpsData))   
                    bitPlot = np.concatenate((bitPlot[N_CYC:DLF*N_CYC],
                                              bitData))                       
                    if noStream % DLF == 0:    
                        plotData(gpsPlot*10,bitPlot*0.2)  
                        if len(FRAME_LIST) > 0:
                            plotText(FRAME_LIST)
                    
                    if codePhase >= 0:
                        COPH_LIST.append((SMP_TIME//NGPS,codePhase))
                                    
                noStream += 1
                    
            await asyncio.sleep(0)
        
    except BaseException:
        printException()
    finally:            
        MEAS_RUNNING = False
        if SAVE_RESULTS:
            saveResults(SAVE_DATE,SAT_NO)
            

# ----------- Stream live data ----------------        

async def streamLive():
    global MEAS_RUNNING
    global STATUS_MSG

    sdr = RtlSdr()
    sdr.set_bias_tee(True)
    sdr.sample_rate = SAMPLE_RATE
    if SDR_FREQCORR != 0:
        sdr.freq_correction = SDR_FREQCORR    
    sdr.center_freq = SDR_CENTERFREQ   
    sdr.gain = SDR_GAIN 

    sdrClosed = False
    measTimeout = False
    loop = asyncio.get_running_loop()
    start_time = loop.time()
    end_time = start_time+MEAS_TIME
    try:
        async for samples in sdr.stream(num_samples_or_bytes=NGPS, 
                                        format='samples'):
            pushToBuffer(samples)
            if loop.time()>end_time:                
                MEAS_RUNNING = False                    
                STATUS_MSG = 'Timeout'
            if not MEAS_RUNNING:
                STATUS_MSG = 'sdr to stop ..'
                await sdr.stop()
                STATUS_MSG = 'sdr stopped'
                sdr.close()
                sdrClosed = True
                STATUS_MSG = 'sdr closed'
    except:    
        STATUS_MSG = 'Exception from sdr.stop'
        printException()
    finally:
        MEAS_RUNNING = False
        if not sdrClosed:
            STATUS_MSG = 'sdr to close ..'
            

# --------------- code for plots ------------------        

# initial values for plot data (real and imag)
xd = np.linspace(0, DLF*N_CYC, DLF*N_CYC, endpoint=False)  # in ms
yd = np.zeros(DLF*N_CYC,dtype=MY_COMPLEX)   # global

fig,(axG,axT) = plt.subplots(1,2,figsize=(10,3.5))
fig.canvas.header_visible = False
fig.canvas.mpl_connect('key_press_event', onKeypress) 
fig.subplots_adjust(bottom=0.2)
                    
plt.subplot(1,2,1)
line1, = axG.plot(xd,yd.real,lw=.5)
line2, = axG.plot(xd,yd.imag,lw=.5)
axG.set_ylim(-0.5,0.5)
axG.set_xlabel('time (ms)')
axG.set_ylabel('amplitude (a.u.)') 

axT.set_axis_off()
W,H = 750,150                                     
xs = np.asarray([0,170,300,400])/W        
y = (H-np.asarray(range(5,175,10)))/H             
MAXL = 14    
dataL = []
keys = ['TIME (UTC)','TOW','SF-ID','SAMPLE TIME']    
for j,key in enumerate(keys):
    axT.text(xs[j],y[0],key)
for i in range(MAXL):
    dataL.append({})
    for j,key in enumerate(keys):
        dataL[i][key] = axT.text(xs[j],y[i+1],'')        
startL = axT.text(0,1.04,'')
        
# --- asyncio tasks -------------

loop = asyncio.get_event_loop()
task1 = loop.create_task(processData())
task2 = loop.create_task(streamLive())

print('Click on graph and press q to exit!')
        
